In [3]:
from functions import download_csv
download_csv()

In [4]:
import pandas as pd
import numpy as np
import warnings

# Ignore all warnings
warnings.filterwarnings("ignore")

# Load csv file of EPL fixtures and results
df = pd.read_csv('EPL_results_2024.csv')
# Splitting the DataFrame based on the 'result' column
results = df.dropna(subset=['Result'])
fixtures = df[df['Result'].isna()]

# Display the first five rows of the df
results.head()

,Match Number,Round Number,Date,Location,Home Team,Away Team,Result
0,1,1,11/08/2023 20:00,Turf Moor,Burnley,Man City,0 - 3
1,2,1,12/08/2023 13:00,Emirates Stadium,Arsenal,Nottingham Forest,2 - 1
2,3,1,12/08/2023 15:00,Vitality Stadium,Bournemouth,West Ham,1 - 1
3,4,1,12/08/2023 15:00,Amex Stadium,Brighton,Luton,4 - 1
4,5,1,12/08/2023 15:00,Goodison Park,Everton,Fulham,0 - 1


In [5]:
# Splitting the 'result' column into 'FTGH' and 'FTGA' (corresponding to full-time goals home and full-time goals away)
results[['FTHG', 'FTAG']] = results['Result'].str.split(' - ', expand=True)

# Converting the 'FTGH' and 'FTGA' columns to integers
results['FTHG'] = results['FTHG'].astype(int)
results['FTAG'] = results['FTAG'].astype(int)

for i, row in results.iterrows():
    if row['FTHG'] > row['FTAG']:
        results.at[i, 'Result'] = 'H'
    elif row['FTHG'] < row['FTAG']:
        results.at[i, 'Result'] = 'A'
    else: 
        results.at[i, 'Result'] = 'D'

# Group by home teams to find average goals for and against for every EPL team home and away
home_teams = results.groupby('Home Team').agg({'FTHG': 'mean', 'FTAG': 'mean', 'Date': 'count'}).rename({'Date':'GP'}, axis=1).reset_index()
away_teams = results.groupby('Away Team').agg({'FTHG': 'mean', 'FTAG': 'mean', 'Date': 'count'}).rename({'Date':'GP'}, axis=1).reset_index()

# Calculate league average goals by thew home and away teams
league_average_home = np.average(home_teams['FTHG'], weights=home_teams['GP'])
league_average_away = np.average(away_teams['FTAG'], weights=away_teams['GP'])

# Caculate offensive and defensive strength ratings for the home and away teams
home_teams['o_strength'] = home_teams['FTHG']/league_average_home
home_teams['d_strength'] = home_teams['FTAG']/league_average_away
away_teams['o_strength'] = away_teams['FTAG']/league_average_away
away_teams['d_strength'] = away_teams['FTHG']/league_average_home

In [6]:
# Change date column to pandas datetime format and sort from soonest fixture onwards
fixtures['Date'] = pd.to_datetime(fixtures['Date'], format='%d/%m/%Y %H:%M')
fixtures.sort_values(by='Date', inplace=True)
fixtures.reset_index(drop=True, inplace=True)
fixtures.head()

,Match Number,Round Number,Date,Location,Home Team,Away Team,Result
0,176,18,2024-02-20 19:30:00,Etihad Stadium,Man City,Brentford,NaN
1,257,26,2024-02-21 19:30:00,Anfield,Liverpool,Luton,NaN
2,255,26,2024-02-23 20:00:00,Stamford Bridge,Chelsea,Spurs,NaN
3,252,26,2024-02-24 15:00:00,Villa Park,Aston Villa,Nottingham Forest,NaN
4,254,26,2024-02-24 15:00:00,Amex Stadium,Brighton,Everton,NaN


In [7]:
from scipy.stats import poisson

# Initialize columns for home expected goals, away expected goals, and probabilities in the fixtures DataFrame
fixtures['home_xg'] = 0
fixtures['away_xg'] = 0
fixtures['home_win%'] = 0
fixtures['away_win%'] = 0
fixtures['draw%'] = 0

# Initialize an empty list to store unique team names
teams = []

# Iterate over each row in the fixtures DataFrame
for i, row in fixtures.iterrows():
    # Check if the home team is already in the list of teams
    if row['Home Team'] in teams:
        # If yes, continue to the next iteration of the loop
        continue
    else:
        # If no, add the home team to the list of teams
        teams.append(row['Home Team'])

    # Check if the away team is already in the list of teams
    if row['Away Team'] in teams:
        # If yes, continue to the next iteration of the loop
        continue
    else:
        # If no, add the away team to the list of teams
        teams.append(row['Away Team'])

    # Find home and away teams o/d strength from results tables
    home_o_strength = home_teams[home_teams['Home Team'] == row['Home Team']]['o_strength'].item()
    home_d_strength = home_teams[home_teams['Home Team'] == row['Home Team']]['d_strength'].item()
    away_o_strength = away_teams[away_teams['Away Team'] == row['Away Team']]['o_strength'].item()
    away_d_strength = away_teams[away_teams['Away Team'] == row['Away Team']]['d_strength'].item()

    # Calculate home and away team expected goals
    home_xg = home_o_strength * away_d_strength * league_average_home
    away_xg = away_o_strength * home_d_strength * league_average_away
    fixtures.loc[i, 'home_xg'] = home_xg
    fixtures.loc[i, 'away_xg'] = away_xg

    # Generate the range of values from 0 to 9
    values = np.arange(10)

    # Calculate the home and away probability distribution using the Poisson PMF
    probabilities_home = poisson.pmf(values, home_xg)
    probabilities_away = poisson.pmf(values, away_xg)

    # Create porbability matrix by broadcasting
    matrix = np.outer(probabilities_home, probabilities_away)

    # Initialize variables to store probabilities
    home_win_prob = 0.0
    away_win_prob = 0.0
    draw_prob = 0.0

    # Iterate over the matrix to calculate probabilities
    for j in range(matrix.shape[0]):
        for k in range(matrix.shape[1]):
            if j > k:
                home_win_prob += matrix[j, k]
            elif j < k:
                away_win_prob += matrix[j, k]
            else:
                draw_prob += matrix[j, k]

    # Assign the calculated probabilities (in percentage) to the corresponding columns in the fixtures DataFrame
    fixtures.at[i, 'home_win%'] = home_win_prob * 100
    fixtures.at[i, 'away_win%'] = away_win_prob * 100
    fixtures.at[i, 'draw%'] = draw_prob * 100

    # Check if the number of unique teams is equal to 20
    if len(teams) == 20:
        # If yes, exit the loop
        break

In [10]:
predictions = fixtures.drop(['Match Number','Location', 'Result'], axis=1)
predictions = predictions[predictions['home_xg'] != 0]
predictions

,Round Number,Date,Home Team,Away Team,home_xg,away_xg,home_win%,away_win%,draw%
0,18,2024-02-20 19:30:00,Man City,Brentford,2.092386,0.726126,69.098101,11.770827,19.124346
1,26,2024-02-21 19:30:00,Liverpool,Luton,3.596288,0.761671,87.393561,4.146169,8.060673
2,26,2024-02-23 20:00:00,Chelsea,Spurs,1.598350,2.178378,28.358640,51.070665,20.560669
3,26,2024-02-24 15:00:00,Aston Villa,Nottingham Forest,2.972931,0.786637,81.108858,6.865539,11.922468
4,26,2024-02-24 15:00:00,Brighton,Everton,1.870070,0.907658,59.993886,17.811889,22.191552
5,26,2024-02-24 15:00:00,Crystal Palace,Burnley,1.378577,1.061261,44.124009,28.907581,26.968195
6,26,2024-02-24 15:00:00,Man Utd,Fulham,1.870070,0.921622,59.649322,18.122013,22.225992
8,26,2024-02-24 20:00:00,Arsenal,Newcastle,2.996906,1.126426,74.931603,11.112230,13.846718
9,26,2024-02-25 13:30:00,Wolves,Sheffield Utd,2.201727,0.930931,66.255709,14.669623,19.064509
